In [1]:
from pyspark.sql import functions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1680859770941_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
import gc
from pyspark.sql.types import BooleanType
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# res

In [3]:
input_bucket = 's3://cs5344-twitter-project'
test_pokemon_damping_sdf = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/test_pokemon_damping').cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
pokemon

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[user_id_str: string]

In [31]:
pokemon = test_pokemon_damping_sdf.select(col("_c0").alias("user_id_str"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|        user_id_str|
+-------------------+
|          130447415|
|         2786677338|
|          307902310|
| 811765148595535872|
|          323091128|
|           29758446|
| 909119962572242944|
|            5162861|
| 987228577522376704|
|1470439914953011204|
|         3095001124|
|1349579108187992071|
|1441557548427927554|
|1441557548427927554|
|          252087695|
|         1149178590|
|1302006102364745728|
|         2597010860|
|             890891|
|1365337673074933765|
+-------------------+
only showing top 20 rows

In [4]:
# test_pokemon_damping_sdf.select("_c0").show()
# id info
from pyspark.sql.functions import collect_set
id_info = set(test_pokemon_damping_sdf.select(collect_set("_c0")).first()[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# user info

In [10]:
input_bucket = 's3://cs5344-twitter-project'
path = '/{}/*.json.gz'
folder_list = ['20220122', '20220123','20220124','20220125', 
               '20220126', '20220127','20220128', '20220129',
               '20220130', '20220131', '20220201', '20220202',
               '20220203', '20220204', '20220205', '20220206',
               '20220207', '20220208', '20220209', '20220210',
               '20220211', '20220212', '20220213', '20220214',
               '20220215']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# for folder in folder_list:
raw = spark.read.json(input_bucket + path.format(folder_list[-1]), allowBackslashEscapingAnyCharacter=True)
# raw.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# for folder in folder_list:
raw = spark.read.json(input_bucket + path.format(folder_list[-1]), allowBackslashEscapingAnyCharacter=True)
# raw.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+--------------------+------------------+------------------------+--------------------+--------------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-----------+-------------+----------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+---------------------+-------------+---------+--------------------+------------------+---------------------+
|contributors|coordinates|          created_at|display_text_range|                entities|   extended_entities|      extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status

'favorited':Nullable. Indicates whether this Tweet has been liked by the authenticating user. Example:"favorited":true
'entities': Entities which have been parsed out of the text of the Tweet. Additionally see Entities in Twitter Objects . Example

In [6]:
raw.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'raw' is not defined
Traceback (most recent call last):
NameError: name 'raw' is not defined



In [ ]:
# verified, followers_count, friends_count, listed_count, favourites_count, statuses_count

In [8]:
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user
# user,The user who posted this Tweet. See User data dictionary for complete list of attributes.
# Example highlighting select attributes:
# The UTC datetime that the user account was created on Twitter. Example:
# statuses_count, The number of Tweets (including retweets) issued by the user
# isted_count, The number of public lists that this user is a member of. Example:
user_df = raw.select(
    col("created_at").alias("tweet_created_at"),
    col("user.id_str").alias("user_id_str"),
    col("user.screen_name").alias("user_twitter_handle"),
    "user.verified",
    "user.followers_count",
    "user.friends_count",
    "user.favourites_count",
    "user.statuses_count",
    "user.created_at"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
user_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['tweet_created_at', 'user_id_str', 'user_twitter_handle', 'verified', 'followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'created_at']

In [13]:
user_df.createOrReplaceTempView("user_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
user_filter = spark.sql("""
select user_id_str,count(1)
from user_df
group by user_id_str
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|        user_id_str|count(1)|
+-------------------+--------+
|         2911177863|       1|
|1468059641313918976|       2|
|          604260054|       1|
|         2791052843|       1|
|          275538641|       1|
|1076305277228597248|       1|
|1476021185322225664|      11|
|1252909103061532672|       1|
| 730410821583441920|       1|
|1469306942862467084|       1|
|1486698341430083585|       5|
|1180832638707228674|       1|
| 759256334554566656|       1|
| 925770679299067905|       4|
|          561566059|       1|
|1466670684344627201|       3|
|1446198790042570759|       1|
|1487143176230129668|       7|
|1072645079376965632|       1|
|1476874579712909314|       9|
+-------------------+--------+
only showing top 20 rows

In [28]:
user_filter = spark.sql("""
select user_id_str, 
        user_twitter_handle, 
        verified, 
        followers_count, 
        friends_count, 
        favourites_count, 
        statuses_count, 
        created_at,
        tweet_created_at,
        rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
from user_df
where user_id_str = '1476021185322225664'
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1476021185322225664|          TanjaPla1|   false|            423|          451|            2665|         19956|Wed Dec 29 02:44:...|Tue Feb 15 21:02:...|          1|
|1476021185322225664|          TanjaPla1|   false|            423|          451|            2651|         19846|Wed Dec 29 02:44:...|Tue Feb 15 20:17:...|          2|
|1476021185322225664|          TanjaPla1|   false|            423|          449|            2643|         19791|Wed Dec 29 02:44:...|Tue Feb 15 19:51:...|          3

In [44]:
user_filter = spark.sql("""
select *
from
(select user_id_str, 
        user_twitter_handle, 
        verified, 
        followers_count, 
        friends_count, 
        favourites_count, 
        statuses_count, 
        created_at,
        tweet_created_at,
        rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
from user_df
where user_id_str is not null) u
where create_rank = 1
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: string, _c1: string, _c2: string]

In [45]:
user_filter.createOrReplaceTempView("user_filter")
pokemon.createOrReplaceTempView("pokemon")

user_join = spark.sql("""
select user_filter.*
from
user_filter
join
pokemon
on user_filter.user_id_str = pokemon.user_id_str""")

user_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1235177017441529861|           game7400|   false|          18430|         2215|            8277|          7677|Wed Mar 04 12:15:...|Tue Feb 15 15:23:...|          1|
|1483999454244274178|       ashward_game|   false|          54211|            4|              17|            27|Thu Jan 20 03:07:...|Tue Feb 15 09:49:...|          1|
|           19426551|                NFL|    true|       28837319|         2723|            3080|        234547|Sat Jan 24 01:28:...|Tue Feb 15 22:55:...|          1

In [41]:
pokemon

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[user_id_str: string]

In [42]:
user_filter

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
user_join.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [48]:
diff = pokemon.join(user_join, on="user_id_str", how="left_anti")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
diff.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|        user_id_str|
+-------------------+
|          130447415|
|         2786677338|
|          307902310|
| 811765148595535872|
|          323091128|
|           29758446|
| 909119962572242944|
|            5162861|
| 987228577522376704|
|1470439914953011204|
|         3095001124|
|1349579108187992071|
|1441557548427927554|
|1441557548427927554|
|          252087695|
|         1149178590|
|         2597010860|
|             890891|
|1365337673074933765|
|1432825944259973123|
+-------------------+
only showing top 20 rows

In [51]:
diff.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

34

In [52]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession

# Define the schema for the empty DataFrame
schema = StructType([
    StructField("user_id_str", StringType(), True),
])

# Create an empty DataFrame with the specified schema
diff = spark.createDataFrame([], schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
input_bucket = 's3://cs5344-twitter-project'
path = '/{}/*.json.gz'
folder_list = ['20220122', '20220123','20220124','20220125', 
               '20220126', '20220127','20220128', '20220129',
               '20220130', '20220131', '20220201', '20220202',
               '20220203', '20220204', '20220205', '20220206',
               '20220207', '20220208', '20220209', '20220210',
               '20220211', '20220212', '20220213', '20220214',
               '20220215']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
input_bucket = 's3://cs5344-twitter-project'
test_pokemon_damping_sdf = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/test_pokemon_damping').cache()
pokemon = test_pokemon_damping_sdf.select(col("_c0").alias("user_id_str"))
pokemon.createOrReplaceTempView("pokemon")
diff = pokemon.alias("diff")
pokemon_inf = None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
for folder in folder_list[::-1]:
# folder=folder_list[-1]
    folder_path = input_bucket + path.format(folder)
    raw = spark.read.json(folder_path, allowBackslashEscapingAnyCharacter=True)

    user_df = raw.select(
        col("user.id_str").alias("user_id_str"),
        col("user.screen_name").alias("user_twitter_handle"),
        "user.verified",
        "user.followers_count",
        "user.friends_count",
        "user.favourites_count",
        "user.statuses_count",
        "user.created_at",
        col("created_at").alias("tweet_created_at"),
        "retweet_count"
    )

    user_df.createOrReplaceTempView("user_df")
    user_info = spark.sql("""
    select  user_id_str, 
            user_twitter_handle, 
            sum(retweet_count) as retweets_count
            from user_df
            group by user_id_str, 
                     user_twitter_handle
    """)
    user_info.createOrReplaceTempView("user_info")

    user_filter = spark.sql("""
    select u_1.*,
            user_info.retweets_count, 
            (user_info.retweets_count + u_1.favourites_count)/u_1.followers_count as IER
    from
        (select *
        from
            (select user_id_str, 
                    user_twitter_handle, 
                    verified, 
                    followers_count, 
                    friends_count, 
                    favourites_count, 
                    statuses_count, 
                    created_at,
                    tweet_created_at,
                    rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
            from user_df
            where user_id_str is not null) u
        where create_rank = 1) u_1
    join user_info
    on u_1.user_id_str = user_info.user_id_str
    and u_1.user_twitter_handle = user_info.user_twitter_handle
    """)
    user_filter.createOrReplaceTempView("user_filter")

    diff.createOrReplaceTempView("diff")
    user_join = spark.sql("""
    select user_filter.*
    from
    user_filter
    join
    diff
    on user_filter.user_id_str = diff.user_id_str""")

    if pokemon_inf:
        pokemon_inf = pokemon_inf.union(user_join)
    else:
        pokemon_inf =user_join.alias("pokemon_inf")
    diff = diff.join(user_join, on="user_id_str", how="left_anti")
    if diff.count() == 0:
        print('okaykkk')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/2/statements/6 with error payload: {"msg":"requirement failed: Session isn't active."}


In [7]:
pokemon_inf.show()

VBox()

An error was encountered:
Session 2 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/07 10:43:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/07 10:43:33 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-41-208.ap-southeast-1.compute.internal/172.31.41.208:8032
23/04/07 10:43:34 INFO Configuration: resource-types.xml not found
23/04/07 10:43:34 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/07 10:43:34 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/07 10:43:34 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/07 10:43:34 INFO Client: Setting up container launch context for our AM
23/04/07 10:43:34 INFO Client: Setting up the launch environment for our AM container
23/04/07 10:43:34 INFO C

In [33]:
pokemon_inf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+--------------+-------------------------------------------------------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|retweets_count|((retweets_count + favourites_count) / followers_count)|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+--------------+-------------------------------------------------------+
|1409917553514848257|        PokemonGems|   false|          48683|          383|            6517|          1041|Tue Jun 29 16:52:...|Tue Feb 15 23:27:...|          1|             0|                                    0.13386603126348007|
|           19426551|                NFL|    tru

In [ ]:
# IER = (retweets + favorites) / followers

In [28]:
# raw.select().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------+
|retweet_count|favorite_count|
+-------------+--------------+
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
+-------------+--------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import sum, first

# Filter DataFrame to include only tweets by a particular influencer
influencer_tweets = tweets_df.filter(tweets_df.user_id == influencer_id)

# Calculate total number of retweets and favorites for the influencer's tweets
total_retweets = influencer_tweets.agg(sum("retweets")).first()[0]
total_favorites = influencer_tweets.agg(sum("favorites")).first()[0]

# Get the number of followers for the influencer
follower_count = influencer_tweets.select("followers").distinct().first()[0]

# Calculate the IER
ier = (total_retweets + total_favorites) / follower_count

In [ ]:
diff.count(), pokemon_inf.count()

In [79]:
pokemon.count() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [67]:
pokemon_inf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1235177017441529861|           game7400|   false|          18430|         2215|            8277|          7677|Wed Mar 04 12:15:...|Tue Feb 15 15:23:...|          1|
|1483999454244274178|       ashward_game|   false|          54211|            4|              17|            27|Thu Jan 20 03:07:...|Tue Feb 15 09:49:...|          1|
|           19426551|                NFL|    true|       28837319|         2723|            3080|        234547|Sat Jan 24 01:28:...|Tue Feb 15 22:55:...|          1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
res.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1235177017441529861|           game7400|   false|          18430|         2215|            8277|          7677|Wed Mar 04 12:15:...|Tue Feb 15 15:23:...|          1|
|1483999454244274178|       ashward_game|   false|          54211|            4|              17|            27|Thu Jan 20 03:07:...|Tue Feb 15 09:49:...|          1|
|           19426551|                NFL|    true|       28837319|         2723|            3080|        234547|Sat Jan 24 01:28:...|Tue Feb 15 22:55:...|          1

In [63]:
res.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [80]:
print("-"*20 + "user_filter" + "-"*20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------------------user_filter--------------------

In [ ]:
def pre_process(folder, diff):
# folder=folder_list[-1]
#     folder_path = input_bucket + path.format(folder)
    raw = spark.read.json(folder, allowBackslashEscapingAnyCharacter=True)

    user_df = raw.select(
        col("user.id_str").alias("user_id_str"),
        col("user.screen_name").alias("user_twitter_handle"),
        "user.verified",
        "user.followers_count",
        "user.friends_count",
        "user.favourites_count",
        "user.statuses_count",
        "user.created_at",
        col("created_at").alias("tweet_created_at"),
        "retweet_count"
    )

    user_df.createOrReplaceTempView("user_df")
    retweet_count = spark.sql("""
    select  user_id_str, 
            user_twitter_handle, 
            sum(retweet_count) as retweets_count
            from user_df
            group by user_id_str, 
                     user_twitter_handle
    """)
    user_info.createOrReplaceTempView("user_info")

    user_filter = spark.sql("""
    select u_1.*,
            user_info.retweets_count, 
            (user_info.retweets_count + u_1.favourites_count)/u_1.followers_count as IER
    from
        (select *
        from
            (select user_id_str, 
                    user_twitter_handle, 
                    verified, 
                    followers_count, 
                    friends_count, 
                    favourites_count, 
                    statuses_count, 
                    created_at,
                    tweet_created_at,
                    rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
            from user_df
            where user_id_str is not null) u
        where create_rank = 1) u_1
    join user_info
    on u_1.user_id_str = user_info.user_id_str
    and u_1.user_twitter_handle = user_info.user_twitter_handle
    """)
    user_filter.createOrReplaceTempView("user_filter")

    diff.createOrReplaceTempView("diff")
    user_join = spark.sql("""
    select user_filter.*
    from
    user_filter
    join
    diff
    on user_filter.user_id_str = diff.user_id_str""")
    
   
    return user_join, diff

